# **Credit card fraude detection**

Ce projet s'inscrit d'une série de projet d'entrainement pour améliorer mes compétences en data science. 

Il consiste à contruire un modèle de machine learning pour prédire les cartes de crédits frauduleuses.

La base de données utilisée ainsi que la description est accessible [ici](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/data).



In [1]:
import pandas as pd

df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')